In [1]:
import boto3

In [2]:
BUCKET = 'dbla-compendium-artifact-bucket'

ddb = boto3.client('dynamodb')
local_ddb = boto3.client('dynamodb', endpoint_url='http://localhost:8000')

In [3]:
tables = ddb.list_tables()['TableNames']
print(tables)


['compendium_cards', 'compendium_crystals', 'compendium_directions', 'compendium_foes', 'compendium_mbox', 'compendium_methods', 'compendium_notecards', 'compendium_questmaster', 'compendium_quests', 'compendium_queststeps', 'compendium_questwords', 'compendium_transactions', 'compendium_users']


In [8]:
for table in tables:
    try:
        response = ddb.update_table(
            TableName=table,
            BillingMode='PAY_PER_REQUEST'
        )
    except Exception as e:
        print(f"Error updating table {table}: {e}")


In [ ]:
for table in tables:

    try:
        response = ddb.update_continuous_backups(
            TableName=table,
            PointInTimeRecoverySpecification={
                'PointInTimeRecoveryEnabled': True
            }
        )
        print("PITR enabled successfully for the table.")
        print(response)
    except Exception as e:
        print(f"Error enabling PITR: {str(e)}")

In [ ]:

for table in tables:
    response = ddb.export_table_to_point_in_time(
        TableArn=f'arn:aws:dynamodb:{boto3.Session().region_name}:{boto3.client("sts").get_caller_identity()["Account"]}:table/{table}',
        S3Bucket=BUCKET,
        S3Prefix=f'ddb_tables/{table}/',
        ExportFormat='DYNAMODB_JSON'
    )
    print(f"Export started for table {table}. Export ARN: {response['ExportDescription']['ExportArn']}")

In [ ]:
# for each table, restore it from S3
for table in tables:
    table_attributes = ddb.describe_table(TableName=table)['Table']['AttributeDefinitions']
    key_schema = ddb.describe_table(TableName=table)['Table']['KeySchema']
    print(table_attributes) 
    print(key_schema)
    response = local_ddb.import_table(
        S3BucketSource={
            'S3Bucket': BUCKET,
            'S3KeyPrefix': f'ddb_tables/'
        },
        InputFormat='DYNAMODB_JSON',
        TableCreationParameters={
            'TableName': table,
            'BillingMode': 'PAY_PER_REQUEST',
            'AttributeDefinitions': table_attributes,
            'KeySchema': key_schema
        },
    )
    print(f"Restored table {table} from S3.")
    print(response)

In [19]:
res = ddb.scan(
    TableName='compendium_cards',
    FilterExpression='card_shortname = :name',
    ExpressionAttributeValues={
        ':name': {'S': 'death'}
    },
    ProjectionExpression='card_death'
)

for item in res['Items']:
    print(item)


{'card_death': {'N': '1'}}
